In [1]:
import folium
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point

In [2]:
land = gpd.read_file('land-shape/ne_10m_land.shp')

In [3]:
def land_check(latitude, longitude):
    point = Point(latitude, longitude)
    return land.contains(point).any()

In [4]:
coordinates = pd.read_csv('coordinates/hexagonal-packing-v3.csv')

In [5]:
bali_map = folium.Map(location=(-8.429957624116502, 115.03252775649212), zoom_start=2)

In [6]:
land_list = []
for index, row in coordinates.iterrows():
    lat, lon = row['lat'], row['lon']
    if land_check(lat, lon):
        folium.Marker([lat, lon], popup='land', icon=folium.Icon(color='green')).add_to(bali_map)
        land_list.append((lat, lon))
    else:
        print(f'{index}coordinates {lat, lon} is not a land')

0coordinates (-8.43778787037442, 115.03706815550647) is not a land
1coordinates (-8.437787765236736, 115.04614895353458) is not a land
2coordinates (-8.437787660099051, 115.05522975156023) is not a land
3coordinates (-8.437787554961368, 115.06431054958344) is not a land
4coordinates (-8.437787449823686, 115.0733913476042) is not a land
5coordinates (-8.437787344686006, 115.08247214562248) is not a land
6coordinates (-8.43778723954833, 115.09155294363832) is not a land
7coordinates (-8.437787134410652, 115.1006337416517) is not a land
8coordinates (-8.437787029272975, 115.10971453966263) is not a land
9coordinates (-8.437786924135303, 115.1187953376711) is not a land
10coordinates (-8.43778681899763, 115.12787613567713) is not a land
11coordinates (-8.43778671385996, 115.13695693368066) is not a land
12coordinates (-8.437786608722291, 115.14603773168176) is not a land
13coordinates (-8.437786503584622, 115.15511852968042) is not a land
14coordinates (-8.437786398446955, 115.164199327676

In [7]:
print(coordinates)

           lat         lon
0    -8.437788  115.037068
1    -8.437788  115.046149
2    -8.437788  115.055230
3    -8.437788  115.064311
4    -8.437787  115.073391
...        ...         ...
1995 -8.743164  115.450581
1996 -8.743164  115.459669
1997 -8.743164  115.468757
1998 -8.743164  115.477845
1999 -8.743164  115.486933

[2000 rows x 2 columns]


In [8]:
bali_map.save('map-export/bali.html')

In [9]:
print(land_check(-8.429957624116502, 115.03252775649212))

False


In [3]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point

input_csv_path = 'coordinates/hexagonal-packing-v3.csv'
df = pd.read_csv(input_csv_path)

print(f"Loaded {len(df)} points from {input_csv_path}")

gdf_points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326")
print(f"Created GeoDataFrame with {len(gdf_points)} points")

land_shapefile_path = 'land-shape/ne_10m_land.shp'
sea_shapefile_path = 'sea-shape/ne_10m_ocean.shp'
gdf_land = gpd.read_file(land_shapefile_path)
gdf_sea = gpd.read_file(sea_shapefile_path)

print(f"Loaded land shapefile from {land_shapefile_path} with {len(gdf_land)} polygons")
print(f"Loaded sea shapefile from {sea_shapefile_path} with {len(gdf_sea)} polygons")

gdf_points = gdf_points.to_crs(gdf_land.crs)

gdf_points_on_land = gpd.sjoin(gdf_points, gdf_land, how='inner', predicate='within')
print(f"Found {len(gdf_points_on_land)} points on land")

gdf_points_in_sea = gpd.sjoin(gdf_points, gdf_sea, how='inner', predicate='within')
print(f"Found {len(gdf_points_in_sea)} points in the sea")

gdf_land_points = gdf_points_on_land[~gdf_points_on_land.index.isin(gdf_points_in_sea.index)]
print(f"Filtered {len(gdf_land_points)} points that are on land and not in the sea")

output_csv_path = 'coordinates/land_points.csv'
gdf_land_points.drop(columns='geometry').to_csv(output_csv_path, index=False)

print(f"Filtered land points saved to {output_csv_path}")


Loaded 14000 points from coordinates/hexagonal-packing-v3.csv
Created GeoDataFrame with 14000 points
Loaded land shapefile from land-shape/ne_10m_land.shp with 11 polygons
Loaded sea shapefile from sea_shape/ne_10m_ocean.shp with 1 polygons
Found 6673 points on land
Found 7327 points in the sea
Filtered 6673 points that are on land and not in the sea
Filtered land points saved to coordinates/land_points.csv


In [6]:
df = pd.read_csv('coordinates/land_points.csv')
bali_map = folium.Map(location=(-8.445613588116192,115.44117199092116), zoom_start=12)
for index, row in df.iterrows():
    folium.Marker([row['lat'], row['lon']], popup='land', icon=folium.Icon(color='green')).add_to(bali_map)
    
bali_map.save('map-export/land_points.html')


In [7]:
SE = (-8.781320, 114.587503)
NW = (-8.564264, 114.387365)

# Isolate around that area 
df = pd.read_csv('coordinates/land_points.csv')
df = df[(df['lat'] < NW[0]) & (df['lat'] > SE[0]) & (df['lon'] > NW[1]) & (df['lon'] < SE[1])]
df.to_csv('coordinates/land_points_NOTbali.csv', index=False)


In [8]:
df = pd.read_csv('coordinates/land_points_NOTbali.csv')
bali_map = folium.Map(location=(-8.445613588116192,115.44117199092116), zoom_start=12)
for index, row in df.iterrows():
    folium.Marker([row['lat'], row['lon']], popup='land', icon=folium.Icon(color='green')).add_to(bali_map)
    
bali_map.save('map-export/land_points_NOTbali.html')

In [10]:
land_points = pd.read_csv('coordinates/land_points.csv')
land_points_NOTbali = pd.read_csv('coordinates/land_points_NOTbali.csv')

merged_df = pd.merge(land_points, land_points_NOTbali, on=['lat', 'lon'], how='outer', indicator=True)

filtered_land_points = merged_df[merged_df['_merge'] == 'left_only']

filtered_land_points = filtered_land_points.drop(columns=['_merge'])

filtered_land_points.to_csv('coordinates/filtered_land_points.csv', index=False)
